In [1]:
!pip install afinn
!pip install vaderSentiment
!pip install sklearn


     |████████████████████████████████| 61kB 2.2MB/s 
  Stored in directory: /root/.cache/pip/wheels/b5/1c/de/428301f3333ca509dcf20ff358690eb23a1388fbcbbde008b2
Successfully built afinn
     |████████████████████████████████| 133kB 2.8MB/s 


In [2]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from afinn import Afinn
import numpy as np
import pandas as pd

from nltk.tokenize import word_tokenize
from nltk.tokenize import TweetTokenizer
from collections import Counter
import itertools

from nltk.corpus import stopwords
import string
from nltk import wordpunct_tokenize

from nltk.stem.lancaster import LancasterStemmer
import matplotlib.pyplot as plt
%matplotlib inline
from PIL import Image
import numpy as np

from wordcloud import WordCloud

from sklearn.preprocessing import MinMaxScaler
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

from sklearn.feature_extraction.text import CountVectorizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
#leggo il dataset con pandas
df = pd.read_csv('reviews_small_timed.csv')

#Convert all reviews in lower case
text = df['text']
df['text'] = list(map(lambda x: x.lower(), text))
df.head()


,productid,userid,profile,helpfulNum,helpfulDen,score,text,time,year
0,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,i have bought several of the vitality canned d...,2011-04-27,2011
1,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,product arrived labeled as jumbo salted peanut...,2012-09-07,2012
2,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,this is a confection that has been around a fe...,2008-08-18,2008
3,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,if you are looking for the secret ingredient i...,2011-06-13,2011
4,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,great taffy at a great price. there was a wid...,2012-10-21,2012


In [5]:
contractions = { 
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he's": "he is",
"how'd": "how did",
"how'll": "how will",
"how's": "how is",
"i'd": "i would",
"i'll": "i will",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'll": "it will",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"must've": "must have",
"mustn't": "must not",
"needn't": "need not",
"oughtn't": "ought not",
"shan't": "shall not",
"sha'n't": "shall not",
"she'd": "she would",
"she'll": "she will",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"that'd": "that would",
"that's": "that is",
"there'd": "there had",
"there's": "there is",
"they'd": "they would",
"they'll": "they will",
"they're": "they are",
"they've": "they have",
"wasn't": "was not",
"we'd": "we would",
"we'll": "we will",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"where'd": "where did",
"where's": "where is",
"who'll": "who will",
"who's": "who is",
"won't": "will not",
"wouldn't": "would not",
"you'd": "you would",
"you'll": "you will",
"you're": "you are"
}

#de-contract words
def clean(text):
  text = text.split()
  new_text = []
  for word in text:
    if word in contractions:
      new_text.append(contractions[word])
    else:
      new_text.append(word)
  text = " ".join(new_text)
  return text
      
clean_text = []
for text in df.text:
   clean_text.append(clean(text))
df['clean_text'] = clean_text
df.head()

,productid,userid,profile,helpfulNum,helpfulDen,score,text,time,year,clean_text
0,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,i have bought several of the vitality canned d...,2011-04-27,2011,i have bought several of the vitality canned d...
1,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,product arrived labeled as jumbo salted peanut...,2012-09-07,2012,product arrived labeled as jumbo salted peanut...
2,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,this is a confection that has been around a fe...,2008-08-18,2008,this is a confection that has been around a fe...
3,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,if you are looking for the secret ingredient i...,2011-06-13,2011,if you are looking for the secret ingredient i...
4,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,great taffy at a great price. there was a wid...,2012-10-21,2012,great taffy at a great price. there was a wide...


In [7]:
#Let's tokenize the reviews using word_tokenize from nltk
df['tokenized'] = df.apply(lambda row: word_tokenize(row['clean_text']), axis=1)
df.head()

,productid,userid,profile,helpfulNum,helpfulDen,score,text,time,year,clean_text,tokenized
0,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,i have bought several of the vitality canned d...,2011-04-27,2011,i have bought several of the vitality canned d...,"[i, have, bought, several, of, the, vitality, ..."
1,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,product arrived labeled as jumbo salted peanut...,2012-09-07,2012,product arrived labeled as jumbo salted peanut...,"[product, arrived, labeled, as, jumbo, salted,..."
2,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,this is a confection that has been around a fe...,2008-08-18,2008,this is a confection that has been around a fe...,"[this, is, a, confection, that, has, been, aro..."
3,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,if you are looking for the secret ingredient i...,2011-06-13,2011,if you are looking for the secret ingredient i...,"[if, you, are, looking, for, the, secret, ingr..."
4,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,great taffy at a great price. there was a wid...,2012-10-21,2012,great taffy at a great price. there was a wide...,"[great, taffy, at, a, great, price, ., there, ..."


In [9]:
#show the tokens
tokenized = df['tokenized']
tokenized

0        [i, have, bought, several, of, the, vitality, ...
1        [product, arrived, labeled, as, jumbo, salted,...
2        [this, is, a, confection, that, has, been, aro...
3        [if, you, are, looking, for, the, secret, ingr...
4        [great, taffy, at, a, great, price, ., there, ...
5        [i, got, a, wild, hair, for, taffy, and, order...
6        [this, saltwater, taffy, had, great, flavors, ...
7        [this, taffy, is, so, good, ., it, is, very, s...
8        [right, now, i, am, mostly, just, sprouting, t...
9        [this, is, a, very, healthy, dog, food, ., goo...
10       [i, do, not, know, if, it, is, the, cactus, or...
11       [one, of, my, boys, needed, to, lose, some, we...
12       [my, cats, have, been, happily, eating, felida...
13       [good, flavor, !, these, came, securely, packe...
14       [the, strawberry, twizzlers, are, my, guilty, ...
15       [my, daughter, loves, twizzlers, and, this, sh...
16       [i, love, eating, them, and, they, are, good, .

In [0]:
#Create a flat list
sentences = (list(itertools.chain(tokenized)))
flat_list = [item for sublist in sentences for item in sublist]

#remove stopwords
stop =stopwords.words('english')
#add stopwords
stop =set(stop)
stop.add("br")
text_tokenized_stop = tokenized.apply(lambda x: [item for item in x if item not in stop])
sentences = (list(itertools.chain(text_tokenized_stop)))
flat_list = [item for sublist in sentences for item in sublist]

#remove puntaction
punctuation = string.punctuation
text_tokenized_stop_punct = text_tokenized_stop.apply(lambda x: [item for item in x if item not in punctuation])
sentences = (list(itertools.chain(text_tokenized_stop_punct)))
flat_list = [item for sublist in sentences for item in sublist]

#apply stemming
lancaster_stemmer = LancasterStemmer()
text_tokenized_stem = text_tokenized_stop_punct.apply(lambda x: [lancaster_stemmer.stem(item) for item in x])

In [13]:
count_vect = CountVectorizer(stop_words=None, lowercase=True, max_features=5000)
bow = count_vect.fit_transform(text_tokenized_stem.apply(lambda x: " ".join(x)))
print(len(text_tokenized_stem))
print(bow.shape)
print(bow.toarray())

100000
(100000, 5000)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [0]:
#let's split into train and test set
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test  = train_test_split(
        bow.toarray(), 
        df.score,
        train_size=0.80, 
        random_state=1234)

In [17]:
#Build the linear classifier using logistic regression
from sklearn.linear_model import LogisticRegression

log_model = LogisticRegression()

#let's train with a fit
log_model = log_model.fit(X=X_train, y=y_train)


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [18]:
#Now we can label the evaluation set
y_pred = log_model.predict(X_test)
print(y_pred)

[5 3 5 ... 5 5 5]


In [19]:
#Now let's measure our performances
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

0.7005


In [20]:
#Confusion matrix
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, y_pred))

[[ 1089   120    83    61   512]
 [  293   180   147   100   443]
 [  153   117   386   262   748]
 [   78    62   172   635  1913]
 [  156    63   115   392 11720]]


In [21]:
from sklearn.metrics import precision_recall_fscore_support
precision, recall, f_score, support = precision_recall_fscore_support(y_test, y_pred)
print(precision)
print(recall)
print(f_score)
print(support)

[0.61560204 0.33210332 0.42746401 0.43793103 0.76421492]
[0.58391421 0.15477214 0.23169268 0.22202797 0.94166801]
[0.59933957 0.2111437  0.30050603 0.29466357 0.84371176]
[ 1865  1163  1666  2860 12446]


In [22]:
precision, recall, f_score, support = precision_recall_fscore_support(y_test, y_pred, average = "macro")
print(precision)
print(recall)
print(f_score)

0.5154630637135746
0.4268150010027381
0.4498729256221311


In [23]:
#La media pesata permette di ottenere risultati migliori, cuciti sul size della classe
precision, recall, f_score, support = precision_recall_fscore_support(y_test, y_pred, average = "weighted")
print(precision)
print(recall)
print(f_score)

0.650519531938951
0.7005
0.6603772900171673
